In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input,  Activation
from keras.layers import  LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import initializers, optimizers, layers
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import pandas as pd
import io 
import re                                  # library for regular expression operations
import string 
!pip install emoji                             # for string operations
import emoji
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import regexp_tokenize   # module for tokenizing strings
from nltk.tokenize import TreebankWordTokenizer
import nltk 
import numpy as np
!pip install WordCloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import layer_utils
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Sequential

     |████████████████████████████████| 133kB 8.9MB/s 


In [14]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [15]:
train_df = pd.read_csv("SmallTwitter.csv")# twitter comments
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


# Clean Data


###Remove @ username in Twitter

In [16]:
def remove_at_user(text):

  """
      Remove @username from tweets
  """
  
  return re.sub("@[A-Za-z0-9]+","",text)

In [17]:
train_df["text_clean"] = train_df["tweet"].apply(lambda x: remove_at_user(x))

###  Contractions 

We use the contractions package to expand the contraction in English such as we'll -> we will or we shouldn't've -> we should not have.

In [13]:
!pip install contractions


     |████████████████████████████████| 327kB 7.5MB/s 
     |████████████████████████████████| 286kB 13.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=a6dbdbc24371086a76d64df31e492e988feda4cbc4ff8024fa17208027c1d8df
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [18]:
import contractions
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: contractions.fix(x))

### Repeat the captial words 
Here we repeat the Captial words to confirm the meaning of this word as the user try to focus on it


In [19]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
train_df['tokenized'] = train_df["text_clean"].apply(word_tokenize)
train_df.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,label,tweet,text_clean,tokenized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can not use becaus...,"[thanks, for, #, lyft, credit, i, can, not, us..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,#model i love you take with you all the time...,"[#, model, i, love, you, take, with, you, all,..."
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation,"[factsguide, :, society, now, #, motivation]"


In [20]:
train_df['text_clean']=train_df['tokenized'].apply(lambda x:  [word  if word.isupper() == False else word+" "+word  for word in x]) 
train_df['text_clean'] = [' '.join(map(str, l)) for l in train_df['text_clean']] # join back to text

In [21]:

train_df.head()

,id,label,tweet,text_clean,tokenized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for # lyft credit i can not use because...,"[thanks, for, #, lyft, credit, i, can, not, us..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,# model i love you take with you all the time ...,"[#, model, i, love, you, take, with, you, all,..."
4,5,0,factsguide: society now #motivation,factsguide : society now # motivation,"[factsguide, :, society, now, #, motivation]"


###  Covert to lower case

In [22]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: x.lower())
display(train_df.head())

,id,label,tweet,text_clean,tokenized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for # lyft credit i can not use because...,"[thanks, for, #, lyft, credit, i, can, not, us..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,# model i love you take with you all the time ...,"[#, model, i, love, you, take, with, you, all,..."
4,5,0,factsguide: society now #motivation,factsguide : society now # motivation,"[factsguide, :, society, now, #, motivation]"


### Remove any URLS

In [23]:
def remove_URL(text):
    """
        Remove URLs from a sample string
    """

    return re.sub(r"https?://\S+|www\.\S+", "", text)

In [24]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_URL(x))

### Remove HTML Tages

In [25]:
def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

In [26]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_html(x))

### Replace the Typos, slang, acronyms or informal abbreviations

In [29]:
def informalAbbreviations_clean(text):
        """
            Other manual text cleaning techniques
        """
        # Typos, slang and other
        sample_typos_slang = {
                                "w/e": "whatever",
                                "usagov": "usa government",
                                "recentlu": "recently",
                                "ph0tos": "photos",
                                "amirite": "am i right",
                                "exp0sed": "exposed",
                                "<3": "love",
                                "luv": "love",
                                "amageddon": "armageddon",
                                "trfc": "traffic",
                                "16yr": "16 year"
                                }

        # Acronyms
        sample_acronyms =  { 
                            "mh370": "malaysia airlines flight 370",
                            "okwx": "oklahoma city weather",
                            "arwx": "arkansas weather",    
                            "gawx": "georgia weather",  
                            "scwx": "south carolina weather",  
                            "cawx": "california weather",
                            "tnwx": "tennessee weather",
                            "azwx": "arizona weather",  
                            "alwx": "alabama weather",
                            "usnwsgov": "united states national weather service",
                            "2mw": "tomorrow"
                            }

        
        # Some common abbreviations 
        sample_abbr = {
                        "$" : " dollar ",
                        "€" : " euro ",
                        "4ao" : "for adults only",
                        "a.m" : "before midday",
                        "a3" : "anytime anywhere anyplace",
                        "aamof" : "as a matter of fact",
                        "acct" : "account",
                        "adih" : "another day in hell",
                        "afaic" : "as far as i am concerned",
                        "afaict" : "as far as i can tell",
                        "afaik" : "as far as i know",
                        "afair" : "as far as i remember",
                        "afk" : "away from keyboard",
                        "app" : "application",
                        "approx" : "approximately",
                        "apps" : "applications",
                        "asap" : "as soon as possible",
                        "asl" : "age, sex, location",
                        "atk" : "at the keyboard",
                        "ave." : "avenue",
                        "aymm" : "are you my mother",
                        "ayor" : "at your own risk", 
                        "b&b" : "bed and breakfast",
                        "b+b" : "bed and breakfast",
                        "b.c" : "before christ",
                        "b2b" : "business to business",
                        "b2c" : "business to customer",
                        "b4" : "before",
                        "b4n" : "bye for now",
                        "b@u" : "back at you",
                        "bae" : "before anyone else",
                        "bak" : "back at keyboard",
                        "bbbg" : "bye bye be good",
                        "bbc" : "british broadcasting corporation",
                        "bbias" : "be back in a second",
                        "bbl" : "be back later",
                        "bbs" : "be back soon",
                        "be4" : "before",
                        "bfn" : "bye for now",
                        "blvd" : "boulevard",
                        "bout" : "about",
                        "brb" : "be right back",
                        "bros" : "brothers",
                        "brt" : "be right there",
                        "bsaaw" : "big smile and a wink",
                        "btw" : "by the way",
                        "bwl" : "bursting with laughter",
                        "c/o" : "care of",
                        "cet" : "central european time",
                        "cf" : "compare",
                        "cia" : "central intelligence agency",
                        "csl" : "can not stop laughing",
                        "cu" : "see you",
                        "cul8r" : "see you later",
                        "cv" : "curriculum vitae",
                        "cwot" : "complete waste of time",
                        "cya" : "see you",
                        "cyt" : "see you tomorrow",
                        "dae" : "does anyone else",
                        "dbmib" : "do not bother me i am busy",
                        "diy" : "do it yourself",
                        "dm" : "direct message",
                        "dwh" : "during work hours",
                        "e123" : "easy as one two three",
                        "eet" : "eastern european time",
                        "eg" : "example",
                        "embm" : "early morning business meeting",
                        "encl" : "enclosed",
                        "encl." : "enclosed",
                        "etc" : "and so on",
                        "faq" : "frequently asked questions",
                        "fawc" : "for anyone who cares",
                        "fb" : "facebook",
                        "fc" : "fingers crossed",
                        "fig" : "figure",
                        "fimh" : "forever in my heart", 
                        "ft." : "feet",
                        "ft" : "featuring",
                        "ftl" : "for the loss",
                        "ftw" : "for the win",
                        "fwiw" : "for what it is worth",
                        "fyi" : "for your information",
                        "g9" : "genius",
                        "gahoy" : "get a hold of yourself",
                        "gal" : "get a life",
                        "gcse" : "general certificate of secondary education",
                        "gfn" : "gone for now",
                        "gg" : "good game",
                        "gl" : "good luck",
                        "glhf" : "good luck have fun",
                        "gmt" : "greenwich mean time",
                        "gmta" : "great minds think alike",
                        "gn" : "good night",
                        "g.o.a.t" : "greatest of all time",
                        "goat" : "greatest of all time",
                        "goi" : "get over it",
                        "gps" : "global positioning system",
                        "gr8" : "great",
                        "gratz" : "congratulations",
                        "gyal" : "girl",
                        "h&c" : "hot and cold",
                        "hp" : "horsepower",
                        "hr" : "hour",
                        "hrh" : "his royal highness",
                        "ht" : "height",
                        "ibrb" : "i will be right back",
                        "ic" : "i see",
                        "icq" : "i seek you",
                        "icymi" : "in case you missed it",
                        "idc" : "i do not care",
                        "idgadf" : "i do not give a damn fuck",
                        "idgaf" : "i do not give a fuck",
                        "idk" : "i do not know",
                        "ie" : "that is",
                        "i.e" : "that is",
                        "ifyp" : "i feel your pain",
                        "IG" : "instagram",
                        "iirc" : "if i remember correctly",
                        "ilu" : "i love you",
                        "ily" : "i love you",
                        "imho" : "in my humble opinion",
                        "imo" : "in my opinion",
                        "imu" : "i miss you",
                        "iow" : "in other words",
                        "irl" : "in real life",
                        "j4f" : "just for fun",
                        "jic" : "just in case",
                        "jk" : "just kidding",
                        "jsyk" : "just so you know",
                        "l8r" : "later",
                        "lb" : "pound",
                        "lbs" : "pounds",
                        "ldr" : "long distance relationship",
                        "lmao" : "laugh my ass off",
                        "lmfao" : "laugh my fucking ass off",
                        "lol" : "laughing out loud",
                        "ltd" : "limited",
                        "ltns" : "long time no see",
                        "m8" : "mate",
                        "mf" : "motherfucker",
                        "mfs" : "motherfuckers",
                        "mfw" : "my face when",
                        "mofo" : "motherfucker",
                        "mph" : "miles per hour",
                        "mr" : "mister",
                        "mrw" : "my reaction when",
                        "ms" : "miss",
                        "mte" : "my thoughts exactly",
                        "nagi" : "not a good idea",
                        "nbc" : "national broadcasting company",
                        "nbd" : "not big deal",
                        "nfs" : "not for sale",
                        "ngl" : "not going to lie",
                        "nhs" : "national health service",
                        "nrn" : "no reply necessary",
                        "nsfl" : "not safe for life",
                        "nsfw" : "not safe for work",
                        "nth" : "nice to have",
                        "nvr" : "never",
                        "nyc" : "new york city",
                        "oc" : "original content",
                        "og" : "original",
                        "ohp" : "overhead projector",
                        "oic" : "oh i see",
                        "omdb" : "over my dead body",
                        "omg" : "oh my god",
                        "omw" : "on my way",
                        "p.a" : "per annum",
                        "p.m" : "after midday",
                        "pm" : "prime minister",
                        "poc" : "people of color",
                        "pov" : "point of view",
                        "pp" : "pages",
                        "ppl" : "people",
                        "prw" : "parents are watching",
                        "ps" : "postscript",
                        "pt" : "point",
                        "ptb" : "please text back",
                        "pto" : "please turn over",
                        "qpsa" : "what happens", #"que pasa",
                        "ratchet" : "rude",
                        "rbtl" : "read between the lines",
                        "rlrt" : "real life retweet", 
                        "rofl" : "rolling on the floor laughing",
                        "roflol" : "rolling on the floor laughing out loud",
                        "rotflmao" : "rolling on the floor laughing my ass off",
                        "rt" : "retweet",
                        "ruok" : "are you ok",
                        "sfw" : "safe for work",
                        "sk8" : "skate",
                        "smh" : "shake my head",
                        "sq" : "square",
                        "srsly" : "seriously", 
                        "ssdd" : "same stuff different day",
                        "tbh" : "to be honest",
                        "tbs" : "tablespooful",
                        "tbsp" : "tablespooful",
                        "tfw" : "that feeling when",
                        "thks" : "thank you",
                        "tho" : "though",
                        "thx" : "thank you",
                        "tia" : "thanks in advance",
                        "til" : "today i learned",
                        "tl;dr" : "too long i did not read",
                        "tldr" : "too long i did not read",
                        "tmb" : "tweet me back",
                        "tntl" : "trying not to laugh",
                        "ttyl" : "talk to you later",
                        "u" : "you",
                        "u2" : "you too",
                        "u4e" : "yours for ever",
                        "utc" : "coordinated universal time",
                        "w/" : "with",
                        "w/o" : "without",
                        "w8" : "wait",
                        "wassup" : "what is up",
                        "wb" : "welcome back",
                        "wtf" : "what the fuck",
                        "wtg" : "way to go",
                        "wtpa" : "where the party at",
                        "wuf" : "where are you from",
                        "wuzup" : "what is up",
                        "wywh" : "wish you were here",
                        "yd" : "yard",
                        "ygtr" : "you got that right",
                        "ynk" : "you never know",
                        "zzz" : "sleeping bored and tired"
                        }
            
        sample_typos_slang_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_typos_slang.keys()) + r')(?!\w)')
        sample_acronyms_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_acronyms.keys()) + r')(?!\w)')
        sample_abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in sample_abbr.keys()) + r')(?!\w)')
        
        text = sample_typos_slang_pattern.sub(lambda x: sample_typos_slang[x.group()], text)
        text = sample_acronyms_pattern.sub(lambda x: sample_acronyms[x.group()], text)
        text = sample_abbr_pattern.sub(lambda x: sample_abbr[x.group()], text)
        
        return text

In [30]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: informalAbbreviations_clean(x))

In [31]:
display(train_df.head())

,id,label,tweet,text_clean,tokenized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for # lyft credit i can not use because...,"[thanks, for, #, lyft, credit, i, can, not, us..."
2,3,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,# model i love you take with you all the time ...,"[#, model, i, love, you, take, with, you, all,..."
4,5,0,factsguide: society now #motivation,factsguide : society now # motivation,"[factsguide, :, society, now, #, motivation]"


In [37]:
!pip install autocorrect
from autocorrect import Speller 

### Spelling Correction

In [38]:
def spelling_correction(text):
  """
    Here apply Textblob to correct the spelling
  """
  repeat_pattern = re.compile(r'(\w)\1*')
  match_substitution = r'\1'
  new_word = repeat_pattern.sub(match_substitution,text)
  spell = Speller(lang='en')  
  return spell(new_word)

In [39]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: spelling_correction(x))

In [50]:
  dict_emot= { ':-)'  : b'\xf0\x9f\x98\x8a'.decode('utf-8'),
                ':)'   : b'\xf0\x9f\x98\x8a'.decode('utf-8'),
                '=)'   : b'\xf0\x9f\x98\x8a'.decode('utf-8'),  # Smile or happy
                ':-D'  : b'\xf0\x9f\x98\x83'.decode('utf-8'),
                ':D'   : b'\xf0\x9f\x98\x83'.decode('utf-8'),
                '=D'   : b'\xf0\x9f\x98\x83'.decode('utf-8'),  # Big smile
                '>:-(' : b'\xF0\x9F\x98\xA0'.decode('utf-8'),
                ':@'   : b'\xF0\x9F\x98\xA0'.decode('utf-8')   # Angry face
                }

### Converting some emojis text to emojis

In [41]:

# def emojiTransalte(Text):

#   dict_emot= { ':-)'  : b'\xf0\x9f\x98\x8a'.decode('utf-8'),
#                 ':)'   : b'\xf0\x9f\x98\x8a'.decode('utf-8'),
#                 '=)'   : b'\xf0\x9f\x98\x8a'.decode('utf-8'),  # Smile or happy
#                 ':-D'  : b'\xf0\x9f\x98\x83'.decode('utf-8'),
#                 ':D'   : b'\xf0\x9f\x98\x83'.decode('utf-8'),
#                 '=D'   : b'\xf0\x9f\x98\x83'.decode('utf-8'),  # Big smile
#                 '>:-(' : b'\xF0\x9F\x98\xA0'.decode('utf-8'),
#                 ':@'   : b'\xF0\x9F\x98\xA0'.decode('utf-8')   # Angry face
#                 }
#   return dict_emot[text]

In [ ]:
# train_df["text_clean"] = train_df["text_clean"].apply(lambda x: emojiTransalte(x))

### Remove non ascii charcters

In [51]:
def remove_non_ascii(text):

    """
        Remove non-ASCII characters 
    """

    return re.sub(r'[^\x00-\x7f]',r'', text) 

In [52]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_non_ascii(x))

### Remove any other special characters

In [53]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [54]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_special_characters(x))

### Remove punctuation

In [56]:
def remove_punct(text):
    """
        Remove the punctuation
    """
    return text.translate(str.maketrans('', '', string.punctuation))

In [57]:
train_df["text_clean"] = train_df["text_clean"].apply(lambda x: remove_punct(x))


# Text Preprocessing


### **Tokenization**

In [58]:
!pip install nltk
from nltk.tokenize import word_tokenize

train_df["tokenized"] = train_df["text_clean"].apply(word_tokenize)
train_df.head()

,id,label,tweet,text_clean,tokenized
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for left credit i can not use because ...,"[thanks, for, left, credit, i, can, not, use, ..."
2,3,0,bihday your majesty,birthday your majesty,"[birthday, your, majesty]"
3,4,0,#model i love u take with u all the time in ...,model i love you take with you al the time in...,"[model, i, love, you, take, with, you, al, the..."
4,5,0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"



### **Remove stop word**

In [59]:
nltk.download("stopwords")
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
train_df['stopwords_removed'] = train_df["tokenized"].apply(lambda x: [word for word in x if word not in stop])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### **Part of Speech Tagging (POS Tagging):**

Part of speech tagging (POS tagging) distinguishes the part of speech (noun, verb, adjective, and etc.) of each word in the text.

In [64]:
import nltk
nltk.download('wordnet')
nltk.download('brown')
from nltk.corpus import wordnet
from nltk.corpus import brown

wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    """
        Create pos_tag with wordnet format
    """
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [65]:
train_df['pos_tag'] = train_df['stopwords_removed'].apply(lambda x: pos_tag_wordnet(x))

### **Lemmatization:**

Lemmatization is the task of determining that two words have the same root, despite their surface differences. The words am, are, and is have the shared lemma be; the words dinner and dinners both have the lemma dinner. 
Lemmatizing each of these forms to the same lemma will let us ﬁnd all mentions of words in Russian like Moscow.

In [68]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [69]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_w_pos'] = train_df['pos_tag'].apply(lambda x: lemmatize_word(x))
train_df['lemmatize_word_w_pos'] = train_df['lemmatize_word_w_pos'].apply(lambda x: [word for word in x if word not in stop]) # double check to remove stop words
train_df['lemmatize_text'] = [' '.join(map(str, l)) for l in train_df['lemmatize_word_w_pos']] # join back to text

### **Saving Dataset**

In [70]:
train_df.to_csv('cleanSmallTwitter.csv')